# Análises Específicas

**Autor:** Leonardo Dantas

**Data:** 09/12/2020

In [2]:
# General
import utils
from utils import *
from datetime import datetime, timedelta

# Data Manipulation
import pandas as pd

# Visualization
import plotly.express as px
import plotly.graph_objects as go

# GIS
import geobr

In [3]:
# Supposed to magically speed up Plotly
import plotly.io as pio
pio.renderers.default = 'iframe'

## Dados

#### Carregar Dados

Todos os dados são carregados a partir do diretório `/data/`.

In [4]:
# Covid Cases by City
cities_cases_file = "br_cities_cases_full.csv"
cities_cases_url = "http://datasource.coronacidades.org/br/cities/cases/full"

cities_cases = load_csv_data(cities_cases_file, cities_cases_url, download_data_anew=False, verbose=True)

Data loaded from local file.


In [5]:
# Covid Cases by State
states_cases_file = "br_states_cases_full.csv"
states_cases_url = "http://datasource.coronacidades.org/br/states/cases/full"

states_cases = load_csv_data(states_cases_file, states_cases_url, download_data_anew=False, verbose=True)

Data loaded from local file.


In [6]:
# Rt Estimates for Brazilian States
states_rt_file = "br_states_rt.csv"
states_rt_file_url = "http://datasource.coronacidades.org/br/states/rt"

states_rt = load_csv_data(states_rt_file, states_rt_file_url, download_data_anew=False, verbose=True)

Data loaded from local file.


In [7]:
# Rt Estimates for Brazilian Cities
cities_rt_file = "br_cities_rt.csv"
cities_rt_file_url = "http://datasource.coronacidades.org/br/cities/rt"

cities_rt = load_csv_data(cities_rt_file, cities_rt_file_url, download_data_anew=False, verbose=True)

Data loaded from local file.


In [8]:
# FarolCovid Cities Data
cities_farolcovid_file = "br_cities_farolcovid_main.csv"
cities_farolcovid_url = "http://datasource.coronacidades.org/br/cities/farolcovid/main"

cities_farolcovid = load_csv_data(cities_farolcovid_file, cities_farolcovid_url, download_data_anew=False, 
                                  verbose=True)

Data loaded from local file.


In [9]:
# FarolCovid States Data
states_farolcovid_file = "br_states_farolcovid_main.csv"
states_farolcovid_url = "http://datasource.coronacidades.org/br/states/farolcovid/main"

states_farolcovid = load_csv_data(states_farolcovid_file, states_farolcovid_url, download_data_anew=False, 
                                  verbose=True)

Data loaded from local file.


In [10]:
# List of Brazilian Capitals
capitals_dict = load_json_data("ibge_capitals.json", verbose=True)
capitals_list = list(capitals_dict.keys())

Data successfully loaded.


#### Datas da Análise

In [11]:
latest_date = states_cases["last_updated"].max()

In [12]:
# Select Current Date
current_date = datetime.strptime(latest_date, "%Y-%m-%d")
#current_date = datetime(2020,11,30)

# Reference Dates
two_weeks_date = current_date - timedelta(days=14)
fifteen_days_date = current_date - timedelta(days=15)
month_date = current_date - timedelta(days=30)

# Sanity Check
print("Current Date: {:%d/%m/%Y}".format(current_date))
print("Two Weeks Ago: {:%d/%m/%Y}".format(two_weeks_date))
print("A Month Ago: {:%d/%m/%Y}".format(month_date))

Current Date: 06/12/2020
Two Weeks Ago: 22/11/2020
A Month Ago: 06/11/2020


## Panorama Nacional

In [13]:
# Consolidate National Data
brazil_cases = states_cases.groupby(["last_updated"], as_index=False).sum()
mavg_days = 7

# Calculate National 7-Day Moving Average of Cases
brazil_cases["daily_cases_mavg"] = brazil_cases["daily_cases"].rolling(mavg_days).mean()

# Calculate National 7-Day Moving Average of Deaths
brazil_cases["new_deaths_mavg"] = brazil_cases["new_deaths"].rolling(7).mean()

### Casos a Nível Nacional

In [14]:
# Lines to Plot
daily_cases_line = ChartLine("Casos Diários", "daily_cases", "skyblue")
daily_cases_mavg_line = ChartLine("Média Móvel de Casos Diários (7 Dias)", "daily_cases_mavg", "royalblue")

cases_lines = [daily_cases_line, daily_cases_mavg_line]

In [15]:
# Markers on the Chart
cases_current_marker = Marker(current_date, "Data de Referência", "last_updated", "daily_cases_mavg", "orangered")
cases_reference_marker = Marker(fifteen_days_date, "15 dias atrás", "last_updated", "daily_cases_mavg", "orangered")

june_cases_marker = Marker(datetime(2020,6,19), "Meados de Junho", "last_updated", "daily_cases_mavg", "orangered")
july_cases_marker = Marker(datetime(2020,7,24), "Final de Julho", "last_updated", "daily_cases_mavg", "orangered")

cases_markers = [cases_current_marker, cases_reference_marker, june_cases_marker, july_cases_marker]

In [16]:
display_line_chart(brazil_cases, "last_updated", cases_lines, cases_markers, title="Casos Diários no Brasil", 
                   x_axis="Data", y_axis="Casos", include_annotations=True, save_HTML=True)

### Mortes a Nível Nacional

In [17]:
# Lines to Plot
daily_deaths_line = ChartLine("Mortes Diárias", "new_deaths", "lightpink")
daily_deaths_mavg_line = ChartLine("Média Móvel de Mortes Diárias (7 Dias)", "new_deaths_mavg", "#EF553B")

deaths_plot_lines = [daily_deaths_line, daily_deaths_mavg_line]

In [18]:
# Markers on the Chart
deaths_current_marker = Marker(current_date, "Data de Referência", "last_updated", "new_deaths_mavg", "royalblue")
deaths_reference_maker = Marker(fifteen_days_date, "15 dias atrás", "last_updated", "new_deaths_mavg", "royalblue")

june_deaths_marker = Marker(datetime(2020,6,19), "Meados de Junho", "last_updated", "new_deaths_mavg", "royalblue")
july_deaths_marker = Marker(datetime(2020,7,24), "Final de Julho", "last_updated", "new_deaths_mavg", "royalblue")

deaths_markers = [deaths_current_marker, deaths_reference_maker]

In [19]:
display_line_chart(brazil_cases, "last_updated", deaths_plot_lines, deaths_markers, 
                   title="Mortes Diárias no Brasil", x_axis="Data", y_axis="Mortes", include_annotations=True, save_HTML=True)

## Transmissibilidade Estimada em Capitais e Estados

### Estados

In [20]:
# Recover State Names
states_names = states_cases[['state_name','state_num_id', 'state_id']]
states_rt_estimates = pd.merge(states_rt, states_names, on="state_num_id")

states_rt_estimates = states_rt_estimates[['state_name', 'state_id', 'Rt_most_likely', 'Rt_most_likely_mavg', 'last_updated']]

In [22]:
# Select RN and SC
rt_RN = states_rt_estimates[ states_rt_estimates['state_id'] == "RN"]
rt_SC = states_rt_estimates[ states_rt_estimates['state_id'] == "SC"]
states_analysis = pd.concat([rt_RN, rt_SC])

In [23]:
fig_analysis_rt = px.line(states_analysis, x="last_updated", y="Rt_most_likely", color="state_name", 
                        title="Evolução da Transmissibilidade nos Estados",
                        labels={"state_name":"Estado", "Rt_most_likely": "Rt Estimado (Mais Provável)", 
                                "last_updated":"Data"})
fig_analysis_rt.add_vline(x=fifteen_days_date)
fig_analysis_rt.show()

In [24]:
fig_analysis_rt.write_html(f"{getcwd()}/Evolução da Transmissibilidade nos Estados.html")